In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
from datasets import Dataset, DatasetDict, Value, Sequence

# 1. Read CSV files
df1 = pd.read_csv("goemotions/data/full_dataset/goemotions_1.csv")
df2 = pd.read_csv("goemotions/data/full_dataset/goemotions_2.csv")
df3 = pd.read_csv("goemotions/data/full_dataset/goemotions_3.csv")

# Combine them if needed (this step depends on how your CSVs are structured)
df = pd.concat([df1, df2, df3], ignore_index=True)

# Inspect
print("Combined dataset size:", len(df))
print(df.head())


Combined dataset size: 211225
                                                text       id  \
0                                    That game hurt.  eew5j0j   
1   >sexuality shouldn’t be a grouping category I...  eemcysk   
2     You do right, if you don't care then fuck 'em!  ed2mah1   
3                                 Man I love reddit.  eeibobj   
4  [NAME] was nowhere near them, he was by the Fa...  eda6yn6   

                author            subreddit    link_id   parent_id  \
0                Brdd9                  nrl  t3_ajis4z  t1_eew18eq   
1          TheGreen888     unpopularopinion  t3_ai4q37   t3_ai4q37   
2             Labalool          confessions  t3_abru74  t1_ed2m7g7   
3        MrsRobertshaw             facepalm  t3_ahulml   t3_ahulml   
4  American_Fascist713  starwarsspeculation  t3_ackt2f  t1_eda65q2   

    created_utc  rater_id  example_very_unclear  admiration  ...  love  \
0  1.548381e+09         1                 False           0  ...     0   
1  1.54808

In [6]:
# Suppose your CSV has a column "labels" that is a list of emotion strings:
# e.g. row["labels"] might be ["joy", "amusement"]

label_groups = {
    "anger": "anger",
    "annoyance": "anger",
    "disgust": "anger",
    "joy": "joy",
    "amusement": "joy",
    "excitement": "joy",
    "sadness": "sadness",
    "grief": "sadness",
    "disappointment": "sadness",
    "love": "love",
    "caring": "love",
    "fear": "fear",
    "nervousness": "fear",
    "surprise": "surprise",
    # ... etc ...
    "admiration": "admiration",
    "approval": "approval",
    "confusion": "confusion",
    "curiosity": "curiosity",
    "desire": "desire",
    "disapproval": "disapproval",
    "embarrassment": "embarrassment",
    "gratitude": "gratitude",
    "optimism": "optimism",
    "pride": "pride",
    "realization": "realization",
    "relief": "relief",
    "remorse": "remorse",
    "neutral": "neutral"
}

# Build a sorted list of unique new labels:
unique_new_labels = sorted(list(set(label_groups.values())))
print("New label set:", unique_new_labels)
# e.g. ['admiration', 'anger', 'approval', 'confusion', 'curiosity', 'desire', 
#       'disapproval', 'embarrassment', 'fear', 'gratitude', 'joy', 'love', 
#       'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 
#       'sadness', 'surprise']

# Create a mapping from label -> index
new_label2id = {lbl: i for i, lbl in enumerate(unique_new_labels)}

def map_labels_to_new(labels_list):
    """
    labels_list is the list of original label strings for a single example.
    We'll map each to the new label, then build a multi-hot vector.
    """
    # Start all zeros
    multi_hot = [0] * len(unique_new_labels)
    for old_lbl in labels_list:
        if old_lbl in label_groups:
            new_lbl = label_groups[old_lbl]
            idx = new_label2id[new_lbl]
            multi_hot[idx] = 1
        # If there's a label not in label_groups, decide how to handle it (ignore or add it)
    return multi_hot


New label set: ['admiration', 'anger', 'approval', 'confusion', 'curiosity', 'desire', 'disapproval', 'embarrassment', 'fear', 'gratitude', 'joy', 'love', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']


In [9]:
label_groups = {
    "anger": "anger",
    "disgust": "anger",   # Merge disgust into "anger"

    "joy": "joy",
    "amusement": "joy",
    "excitement": "joy",

    "sadness": "sadness",
    "grief": "sadness",
    "disappointment": "sadness",

    "love": "love",
    "caring": "love",

    "fear": "fear",
    "nervousness": "fear",

    # Keep some as-is:
    "admiration": "admiration",
    "approval":   "approval",
    "confusion":  "confusion",
    "curiosity":  "curiosity",
    "desire":     "desire",
    "disapproval":"disapproval",
    "embarrassment":"embarrassment",
    "gratitude":  "gratitude",
    "optimism":   "optimism",
    "pride":      "pride",
    "realization":"realization",
    "relief":     "relief",
    "remorse":    "remorse",
    "sadness":    "sadness",
    "surprise":   "surprise",
    "neutral":    "neutral"
}


In [14]:
unique_new_labels = sorted(set(label_groups.values()))
print("New label set:", unique_new_labels)
new_label2id = {lbl: i for i, lbl in enumerate(unique_new_labels)}


New label set: ['admiration', 'anger', 'approval', 'confusion', 'curiosity', 'desire', 'disapproval', 'embarrassment', 'fear', 'gratitude', 'joy', 'love', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']


In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
from datasets import Dataset, DatasetDict, Sequence, Value
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# 1) Read your local CSV
# If you have multiple CSVs (goemotions_1.csv, goemotions_2.csv, etc.), concatenate them
df = pd.read_csv("goemotions/data/full_dataset/goemotions_1.csv")
# df2 = pd.read_csv("goemotions/data/full_dataset/goemotions_2.csv")
# df3 = pd.read_csv("goemotions/data/full_dataset/goemotions_3.csv")
# df = pd.concat([df, df2, df3], ignore_index=True)

print("Columns in df:", df.columns)
print("Dataset size:", len(df))

# 2) List of emotion columns (one column per emotion, with 0/1 values).
#    Make sure these match EXACTLY the columns in your DataFrame.
emotions = [
    "admiration", "amusement", "anger", "approval", "caring",
    "confusion", "curiosity", "desire", "disappointment", "disapproval",
    "disgust", "embarrassment", "excitement", "fear", "gratitude",
    "grief", "joy", "love", "nervousness", "optimism", "pride",
    "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

# 3) Define how to merge overlapping emotions:
#    Key = original emotion column, Value = your new "merged" label name.
label_groups = {
    # Merge 'disgust' into 'anger'
    "anger": "anger",
    "disgust": "anger",

    # Merge 'amusement' & 'excitement' into 'joy'
    "joy": "joy",
    "amusement": "joy",
    "excitement": "joy",

    # Merge 'grief' & 'disappointment' into 'sadness'
    "sadness": "sadness",
    "grief": "sadness",
    "disappointment": "sadness",

    # Merge 'caring' into 'love'
    "love": "love",
    "caring": "love",

    # Merge 'nervousness' into 'fear'
    "fear": "fear",
    "nervousness": "fear",

    # Keep everything else as is
    "admiration": "admiration",
    "approval": "approval",
    "confusion": "confusion",
    "curiosity": "curiosity",
    "desire": "desire",
    "disapproval": "disapproval",
    "embarrassment": "embarrassment",
    "gratitude": "gratitude",
    "optimism": "optimism",
    "pride": "pride",
    "realization": "realization",
    "relief": "relief",
    "remorse": "remorse",
    "neutral": "neutral",
    "surprise": "surprise"
}

# Build a sorted list of your new merged labels
unique_new_labels = sorted(set(label_groups.values()))
print("New label set:", unique_new_labels)

# Map each merged label to an index
new_label2id = {lbl: i for i, lbl in enumerate(unique_new_labels)}


Columns in df: Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')
Dataset size: 70000
New label set: ['admiration', 'anger', 'approval', 'confusion', 'curiosity', 'desire', 'disapproval', 'embarrassment', 'fear', 'gratitude', 'joy', 'love', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']


In [23]:
def row_to_new_multi_hot(row):
    # multi_hot has length = number of merged labels
    multi_hot = [0]*len(unique_new_labels)
    # For each original emotion column
    for emo in emotions:
        # If this row has emo=1, we set the merged label index to 1
        if row[emo] == 1:
            merged_label = label_groups[emo]  # e.g. "anger" if emo is "disgust"
            idx = new_label2id[merged_label]
            multi_hot[idx] = 1
    return multi_hot

# Apply the function to each row
df["new_multi_hot"] = df.apply(row_to_new_multi_hot, axis=1)


  7%|▋         | 7186/105615 [03:30<43:04, 38.09it/s]

In [24]:
# 80% train, 10% val, 10% test
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

print("Train size:", len(df_train))
print("Val size:", len(df_val))
print("Test size:", len(df_test))


Train size: 56000
Val size: 7000
Test size: 7000


In [25]:
ds_train = Dataset.from_pandas(df_train)
ds_val   = Dataset.from_pandas(df_val)
ds_test  = Dataset.from_pandas(df_test)

dataset = DatasetDict({
    "train": ds_train,
    "validation": ds_val,
    "test": ds_test
})


In [26]:
model_checkpoint = "distilbert-base-uncased"  # a smaller model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    tok = tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=128
    )
    # Attach the new_multi_hot vector as labels
    tok["labels"] = examples["new_multi_hot"]
    return tok

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Cast the labels to float for multi-label classification
encoded_dataset = encoded_dataset.cast_column("labels", Sequence(Value("float32")))
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


c:\Second_Sem\csc392\env_py310\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Casting the dataset: 100%|██████████| 7000/7000 [00:00<00:00, 422204.89 examples/s]


In [27]:
num_labels = len(unique_new_labels)
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)
model.config.problem_type = "multi_label_classification"

# Compute pos_weight if desired
def compute_pos_weight(dataset_split, n_labels):
    counts = np.zeros(n_labels)
    for example in dataset_split:
        counts += np.array(example["labels"])
    total_samples = len(dataset_split)
    # pos_weight = (N - count) / count
    pos_weight = (total_samples - counts) / (counts + 1e-5)
    return torch.tensor(pos_weight, dtype=torch.float32)

pos_weight = compute_pos_weight(encoded_dataset["train"], num_labels)

# Custom Trainer to use BCEWithLogitsLoss + pos_weight
class WeightedBCELossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]
        outputs = model(**inputs)
        logits = outputs.logits
        bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(logits.device))
        loss = bce(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define a metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    subset_accuracy = np.mean(np.all(preds == labels, axis=1))
    micro_f1 = f1_score(labels, preds, average="micro", zero_division=0)
    macro_f1 = f1_score(labels, preds, average="macro", zero_division=0)
    return {
        "subset_accuracy": subset_accuracy,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    output_dir="./distilbert_merged_emotions",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=1,
    fp16=True  # if your GPU supports it
)

trainer = WeightedBCELossTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=DefaultDataCollator(),
    compute_metrics=compute_metrics,
)

trainer.train()


c:\Second_Sem\csc392\env_py310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 106/105615 [00:05<45:59, 38.24it/s] 

{'loss': 1.3005, 'learning_rate': 9.46790380609733e-08, 'epoch': 0.0}


  0%|          | 207/105615 [00:08<47:13, 37.20it/s]

{'loss': 1.3277, 'learning_rate': 1.893580761219466e-07, 'epoch': 0.01}


  0%|          | 303/105615 [00:10<47:27, 36.98it/s]

...

{'loss': 1.1562, 'learning_rate': 6.622798712365083e-06, 'epoch': 0.33}


  7%|▋         | 7103/105615 [03:14<44:07, 37.22it/s]

{'loss': 1.047, 'learning_rate': 6.7174777504260555e-06, 'epoch': 0.34}


  7%|▋         | 7183/105615 [03:17<43:04, 38.09it/s]

KeyboardInterrupt: 